In [229]:
import os
import numpy as np
import gensim
import nltk
import math
import string
from scipy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem.porter import *
from collections import Counter
from gensim.models import TfidfModel
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [230]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LAB_JennyChiou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LAB_JennyChiou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [226]:
#retrained model
def retrain(data_file, old_model_file, new_model_file):
    sents = 'You say that you|re restless, you say that you know me too well You|ve seen all my best and you|ve heard all the stories I tell You think you|ve been taken for granted, you|re probably right Stil I remember a November Night, When the dawn on your doorway shone white with frost And the soft love that always began with the touch of your hand And recall the mornings that tossed your hair in the wind. Time|s made it meaningless, I|m insecure, you can tell But why am I leaving unless time had only meant well? There|s nothing left now to excite you, no reason to try. Still I remember a candlelit sky And the summer surrounding the ground where you and I lay And though we were always alone with our secrets known, We both were aware and afraid our closeness might end. My love|s like a dancer, she weaves through the danger complete Her well rehearsed answers with her rational reasons for feet. And if it decreases or ceases to always seem right I|ll just remember a November Night When the dawn on your doorway shone white with frost And the soft love that always began with the touch of your hand And recall the mornings that tossed your hair in the wind.'
    model = Word2Vec.load(old_model_file)
    model.build_vocab(sents, update=True)
    model.train(sents, total_examples=model.corpus_count, epochs=model.iter)
    model.save(new_model_file)
#retrain('temp/','temp/w2vm0719-18','temp/w2vm0719-19')

C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [231]:
model = gensim.models.Word2Vec.load('temp/w2vm0719-19')

In [3]:
'''
#https://zhuanlan.zhihu.com/p/39784226
gensim_model = KeyedVectors.load_word2vec_format('C:/Users/LAB_JennyChiou/Desktop/codeTEMP/GoogleNews-vectors-negative300.bin', binary=True, limit=300000)
print(gensim_model['medicine'])
print('DIM:',len(gensim_model['medicine']))
'''

C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [55]:
#分詞
def get_tokens(text):
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lowers.translate(remove_punctuation_map)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

In [56]:
#過濾缺乏實際意義的 the, a, and 等詞
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [62]:
def tf(word, count):
    return count[word] / sum(count.values())

def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

def idf(word, count_list):
    return math.log(len(count_list) / (1+n_containing(word, count_list)))

def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [228]:
import csv
import pandas as pd
df = pd.read_csv('lyrics_data/survey_tfidf.csv',encoding='ISO-8859-15')
a = df.iat[19,3]
b = a.replace('\n',' ').replace("'",'|')
print(b)
tokens1 = get_tokens(b)
filtered1 = [w for w in tokens1 if not w in stopwords.words('english')]
count1_2 = Counter(filtered1)
stemmer = PorterStemmer()
stemmed1 = stem_tokens(filtered1, stemmer)

You say that you|re restless, you say that you know me too well You|ve seen all my best and you|ve heard all the stories I tell You think you|ve been taken for granted, you|re probably right Stil I remember a November Night, When the dawn on your doorway shone white with frost And the soft love that always began with the touch of your hand And recall the mornings that tossed your hair in the wind. Time|s made it meaningless, I|m insecure, you can tell But why am I leaving unless time had only meant well? There|s nothing left now to excite you, no reason to try. Still I remember a candlelit sky And the summer surrounding the ground where you and I lay And though we were always alone with our secrets known, We both were aware and afraid our closeness might end. My love|s like a dancer, she weaves through the danger complete Her well rehearsed answers with her rational reasons for feet. And if it decreases or ceases to always seem right I|ll just remember a November Night When the dawn on

In [127]:
countlist = [count1_2]
num = 0 
for i, count in enumerate(countlist):
    scores = {word: tfidf(word, count, countlist) for word in count}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    word_list = []
    for word, score in sorted_words[:30]:
        #print("Word: {}, TF-IDF: {}".format(word, round(score, 5)))
        word_list.append(word)
        num += 1
    print('個數：',num)

個數： 30


In [256]:
import csv
import pandas as pd
df = pd.read_csv('lyrics_data/survey_top30tfidf.csv',encoding='ISO-8859-15')
w = df.iat[4,30]
print(w)
wv = model[w]
print(wv)

cares
[ 0.5153681  -0.03074113  0.57268345  0.2090781  -0.20745821 -0.73549056
  0.32369652  0.10069694 -0.37656263 -0.0242989  -0.3280843   0.10875321
 -0.14661033 -0.49504414  0.40372178 -0.32027507 -0.28234705  0.3248457
 -0.39356172  0.14657363  0.69562256  0.15091334 -0.16054916  0.06096634
  0.0935389   0.09999585 -0.05043705  0.48637643 -0.240908    0.00425915
 -0.46895027  0.13853382 -0.4084685  -0.21005368 -0.24950935 -0.005965
  0.34955764 -0.398492    0.18161377  0.6489171   0.12037376  1.2115277
  0.33759528 -0.34108362  0.12027232  0.18619247  0.07626042 -0.44276768
  0.07676855  0.2024285  -0.65003973  0.05974188  0.08652122 -0.21973972
 -0.30787706 -0.17001924  0.48814476  0.86647254 -0.6593732  -0.579492
 -0.2040203  -0.3768105  -0.6152671  -0.297578   -0.43017772 -0.5805448
  0.22450903  0.26915506 -0.23654573 -0.05448423 -0.18046914  1.2481549
 -0.45116597  0.55944735 -0.73816526  0.46470362  0.3537246   0.09223186
  0.29996872 -0.0629668  -0.4662817   0.25864533  0.4

C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [128]:
#挑選TFIDF前30個詞語
with open ('lyrics_data/survey_top30tfidf.csv','a',newline='') as csvfile:
    writer = csv.writer(csvfile)
    #writer.writerow(['track_id','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29'])
    df = pd.read_csv('lyrics_data/survey_tfidf.csv',encoding='ISO-8859-15')
    track_id = df['trackID'][19]
    w0 = word_list[0]
    w1 = word_list[1]
    w2 = word_list[2]
    w3 = word_list[3]
    w4 = word_list[4]
    w5 = word_list[5]
    w6 = word_list[6]
    w7 = word_list[7]
    w8 = word_list[8]
    w9 = word_list[9]
    w10 = word_list[10]
    w11 = word_list[11]
    w12 = word_list[12]
    w13 = word_list[13]
    w14 = word_list[14]
    w15 = word_list[15]
    w16 = word_list[16]
    w17 = word_list[17]
    w18 = word_list[18]
    w19 = word_list[19]
    w20 = word_list[20]
    w21 = word_list[21]
    w22 = word_list[22]
    w23 = word_list[23]
    w24 = word_list[24]
    w25 = word_list[25]
    w26 = word_list[26]
    w27 = word_list[27]
    w28 = word_list[28]
    w29 = word_list[29]
    Table = [[track_id,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29]]
    writer.writerows(Table)
print('DONE')

DONE


In [232]:
#挑選TFIDF前30個詞語的向量
import csv
import pandas as pd
df = pd.read_csv('lyrics_data/survey_top30tfidf.csv',encoding='ISO-8859-15')

with open('lyrics_data/survey_top30tfidf_v.csv', "w", newline='',encoding='ISO-8859-15') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29'])
    for i in range(0,20):

        w1 = df.iat[i,1]
        w2 = df.iat[i,2]
        w3 = df.iat[i,3]
        w4 = df.iat[i,4]
        w5 = df.iat[i,5]
        w6 = df.iat[i,6]
        w7 = df.iat[i,7]
        w8 = df.iat[i,8]
        w9 = df.iat[i,9]
        w10 = df.iat[i,10]
        w11 = df.iat[i,11]
        w12 = df.iat[i,12]
        w13 = df.iat[i,13]
        w14 = df.iat[i,14]
        w15 = df.iat[i,15]
        w16 = df.iat[i,16]
        w17 = df.iat[i,17]
        w18 = df.iat[i,18]
        w19 = df.iat[i,19]
        w20 = df.iat[i,20]
        w21 = df.iat[i,21]
        w22 = df.iat[i,22]
        w23 = df.iat[i,23]
        w24 = df.iat[i,24]
        w25 = df.iat[i,25]
        w26 = df.iat[i,26]
        w27 = df.iat[i,27]
        w28 = df.iat[i,28]
        w29 = df.iat[i,29]
        w30 = df.iat[i,30]
        
        wv1 = model[w1]
        wv2 = model[w2]
        wv3 = model[w3]
        wv4 = model[w4]
        wv5 = model[w5]
        wv6 = model[w6]
        wv7 = model[w7]
        wv8 = model[w8]
        wv9 = model[w9]
        wv10 = model[w10]
        wv11 = model[w11]
        wv12 = model[w12]
        wv13 = model[w13]
        wv14 = model[w14]
        wv15 = model[w15]
        wv16 = model[w16]
        wv17 = model[w17]
        wv18 = model[w18]
        wv19 = model[w19]
        wv20 = model[w20]
        wv21 = model[w21]
        wv22 = model[w22]
        wv23 = model[w23]
        wv24 = model[w24]
        wv25 = model[w25]
        wv26 = model[w26]
        wv27 = model[w27]
        wv28 = model[w28]
        wv29 = model[w29]
        wv30 = model[w30]
        
        Table = [[wv1,wv2,wv3,wv4,wv5,wv6,wv7,wv8,wv9,wv10,wv11,wv12,wv13,wv14,wv15,wv16,wv17,wv18,wv19,wv20,wv21,wv22,wv23,wv24,wv25,wv26,wv27,wv28,wv29,w30]]
        writer.writerows(Table)

C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\LAB_JennyChiou\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() inste